<a href="https://colab.research.google.com/github/malowana/NLP_budzet_obywatelski/blob/main/BO_analiza_morfologiczna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U spacy

  Using cached spacy-3.2.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.0 MB)
  Using cached srsly-2.4.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (451 kB)
  Using cached catalogue-2.0.6-py3-none-any.whl (17 kB)
  Using cached thinc-8.0.13-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (628 kB)
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.5
    Uninstalling thinc-7.4.5:
      Successfully uninstalled thinc-7.4.5
  Attempting uninstall: spacy
    Found existing installation: spacy 2.3.7
    Uninstalling spacy-2.3.7:
      Successfully uninstalled spacy-2.3.7
ERROR: pip's dependency resolver does not currently t

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from gensim.utils import simple_preprocess
from collections import Counter
import gc

import spacy
from spacy.lang.pl.stop_words import STOP_WORDS

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', 'SelectableGroups dict interface')

In [4]:
!python -m spacy download pl_core_news_lg

     |████████████████████████████████| 612.1 MB 12 kB/s 
  Attempting uninstall: pl-core-news-lg
    Found existing installation: pl-core-news-lg 2.3.0
    Uninstalling pl-core-news-lg-2.3.0:
      Successfully uninstalled pl-core-news-lg-2.3.0
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_lg')


In [5]:
nlp = spacy.load('pl_core_news_lg')

# Przygotowanie danych

In [6]:
df = pd.read_excel('/content/drive/My Drive/Konkursy Kaggle/Budżet obywatelski/budzet_obywatelski.xlsx', sheet_name='Arkusz1')

In [7]:
df.rename(columns={'rok ': 'rok', 'miasto ':'miasto'}, inplace=True)

In [8]:
# Usunięcie wartości pustych
df = df[~df['nazwa'].isna()]

In [9]:
df['miasto'] = df['miasto'].map(lambda x: x.upper())

In [10]:
# Usunięcie pustych wpisów

# Indeksy do usunięcia
rem_index = df[df['nazwa'].isnull()].index.tolist()
rem_index += df[df['nazwa']=='bd'].index.tolist()

df.drop(df.index[rem_index], inplace=True)

In [11]:
# Usunięcie duplikatów
df = df.drop_duplicates()

In [12]:
#usuniecie kolumny wszystko
if 'WSZYSTKO' in df: del df['WSZYSTKO']

In [13]:
df.sample(10)

,rok,miasto,nazwa
13815,2019,ŁÓDŹ,"„Z angielskim na co dzień"" - doposażenie PM 7 ..."
13394,2019,ŁÓDŹ,"Wyposażenie Ośrodka Wypoczynkowego „Arturówek""..."
593,2019,GDYNIA,Puchar W biegach dla dzieci i mtodziezy W dzie...
22285,2020,WARSZAWA,Przejście dla pieszych na ul. Terespolskiej pr...
23382,2020,KRAKÓW,Kolorowo i bezpiecznie
247,2020,GDYNIA,Organizacja biegowej imprezy w dzielnicy Mały ...
3844,2020,KATOWICE,X. Dunikowskiego - kwiecista uliczka na Koszutce
22033,2020,WARSZAWA,Festiwal Muzyki sPokoju im. W. Landowskiej
9302,2020,WROCŁAW,Maślicki Ogród Wodny „SREBRNA MAGIA” = Nasadze...
21412,2019,WARSZAWA,"Remont łazienek w budynku hali sportowej ""Saska"""


In [14]:
df.loc[(df['miasto'] == 'WARSZAWA'), "czy_WAW"] = 1
df.loc[df["czy_WAW"].isnull(),  "czy_WAW"] = 0

In [16]:
df["czy_WAW"].value_counts()

0.0    18431
1.0     4599
Name: czy_WAW, dtype: int64

In [17]:
df_waw = df[df['miasto'] == 'WARSZAWA']
df_waw.head()

,rok,miasto,nazwa,czy_WAW
18418,2018,WARSZAWA,Domki dla jeży,1.0
18419,2018,WARSZAWA,Wolskie lato filmowe,1.0
18420,2018,WARSZAWA,Chronimy jerzyki i wróble na Woli - skrzynki l...,1.0
18421,2018,WARSZAWA,Czysta Wisła na Pradze Południe,1.0
18422,2018,WARSZAWA,Wola na 2 koła! Kontraruch rowerowy w całej dz...,1.0


In [28]:
docs_waw = ' '.join(df_waw.nazwa)

In [29]:
docs_waw

'Domki dla jeży Wolskie lato filmowe Chronimy jerzyki i wróble na Woli - skrzynki lęgowe Czysta Wisła na Pradze Południe Wola na 2 koła! Kontraruch rowerowy w całej dzielnicy Bielańska plaża nad Wisłą Chronimy jerzyki i wróble na Pradze -Południe - skrzynki lęgowe leczenie i sterylizacja wolno żyjących kotów Ćwiczenia dla seniorów na otwartych siłowniach Rozwijamy wolskie maluchy Skarby Woli. Czego nie wiesz o swojej dzielnicy? Chronimy jerzyki i wróble na Bielanach - skrzynki lęgowe Czyste powietrze dla Targówka - nasadzenia drzew Tai Chi na świeżym powietrzu Ogródek przy przystanku autobusowym "Antka Rozpylacza 01" Wiem czym oddycham! - tablica informująca o poziomie zanieczyszczenia powietrza (Praga-Południe) Virtualna Warszawa – turystyka dostępna dla wszystkich, również niepełnosprawnych. Ratujemy drzewa Parku Skaryszewskiego - wiązania elastyczne w koronach drzew na podstawie ekspertyzy dendrologicznej. Serek - Miejsce spotkań i relaksu przy metrze Słodowiec Wodny plac zabaw dla 

In [30]:
doc = nlp(docs_waw)

In [31]:
for token in doc:
    print("{:20}{:20}{:10}{:10}{:3}{:3}{:3}".\
          format(token.text, token.lemma_, token.pos_, token.tag_, token.is_stop, token.is_punct, token.is_digit))

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
dla                 dla                 ADP       PREP        1  0  0
rowerów             rower               NOUN      SUBST       0  0  0
w                   w                   ADP       PREP        1  0  0
zrewitalizowanej    zrewitalizowanej    ADJ       PPAS        0  0  0
hydroforni          hydrofornia         NOUN      SUBST       0  0  0
wraz                wraz                ADV       ADV         0  0  0
z                   z                   ADP       PREP        1  0  0
ogródkiem           ogródek             NOUN      SUBST       0  0  0
miododajnym         miododajnym         NOUN      ADJ         0  0  0
                                        SPACE     _SP         0  0  0
Historyczna         historyczny         ADJ       ADJ         0  0  0
Ścieżka             ścieżek             NOUN      SUBST       0  0  0
Edukacyjna          edukacyjny          ADJ       ADJ         0  0  0
Z                   z    